<a href="https://colab.research.google.com/github/cbeckler/amazon_vine_analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu 

In [2]:
# Start Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|                Y|
|R2F0POU5K6F73F|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|          5|            0|          0|   N|                Y|
| RNQOY62705W1K|          4|            0|          0|   N|     

In [5]:
vine_filtered = vine_df.filter("total_votes>=20")

vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R3GSK9MM8DNOYI|          1|            4|         32|   N|     

In [6]:
vine_filtered = vine_filtered.filter("helpful_votes/total_votes >= 0.5")

vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|     

In [14]:
from pyspark.sql.functions import expr

five_star_col = expr("IF(star_rating = 5, 1, 0)")

vine_filtered = vine_filtered.withColumn('five_star', five_star_col)

vine_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|five_star|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|        0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|        1|
|R10LZVBLQHBVJ0|          2|    

In [15]:
vine_yes = vine_filtered.filter("vine == 'Y'")

vine_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|five_star|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
|R3KKUSGFZWSUIY|          5|           56|         63|   Y|                N|        1|
|R10FO5UKKVZBK2|          3|           23|         23|   Y|                N|        0|
| RM4KSGEOR7MU1|          5|           19|         24|   Y|                N|        1|
| RG7VRMYLEXD23|          4|           22|         26|   Y|                N|        0|
|R11O4YSCPSNL6L|          3|           20|         26|   Y|                N|        0|
|R286MFBAJ8NPD6|          5|           46|         51|   Y|                N|        1|
|R1JRR530H4COA2|          5|           22|         28|   Y|                N|        1|
| RQ5WD90PUNBU9|          5|           21|         24|   Y|                N|        1|
|R12648VHCQWUV9|          4|    

In [16]:
vine_no = vine_filtered.filter("vine == 'N'")

vine_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|five_star|
+--------------+-----------+-------------+-----------+----+-----------------+---------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|        0|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|        0|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|        0|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|        0|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|        0|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|        0|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|        0|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|        1|
|R10LZVBLQHBVJ0|          2|    

In [20]:
yes_summary = vine_yes.agg({'star_rating':'count', 'five_star':'sum'})

yes_summary = yes_summary.withColumnRenamed('sum(five_star)','five_star_ratings')

yes_summary = yes_summary.withColumnRenamed('count(star_rating)','all_ratings')

yes_summary = yes_summary.withColumn('percent_five_star', yes_summary['five_star_ratings']/yes_summary['all_ratings']*100)

yes_summary = yes_summary.select(['all_ratings', 'five_star_ratings', 'percent_five_star'])

yes_summary.show()

+-----------+-----------------+-----------------+
|all_ratings|five_star_ratings|percent_five_star|
+-----------+-----------------+-----------------+
|         94|               48|51.06382978723404|
+-----------+-----------------+-----------------+



In [21]:
no_summary = vine_no.agg({'star_rating':'count', 'five_star':'sum'})

no_summary = no_summary.withColumnRenamed('sum(five_star)','five_star_ratings')

no_summary = no_summary.withColumnRenamed('count(star_rating)','all_ratings')

no_summary = no_summary.withColumn('percent_five_star', no_summary['five_star_ratings']/no_summary['all_ratings']*100)

no_summary = no_summary.select(['all_ratings', 'five_star_ratings', 'percent_five_star'])

no_summary.show()

+-----------+-----------------+------------------+
|all_ratings|five_star_ratings| percent_five_star|
+-----------+-----------------+------------------+
|      40471|            15663|38.701786464381904|
+-----------+-----------------+------------------+



In [23]:
summary = vine_filtered.groupby('vine').agg({'star_rating':'count', 'five_star':'sum'})

summary = summary.withColumnRenamed('sum(five_star)','five_star_ratings')

summary = summary.withColumnRenamed('count(star_rating)','all_ratings')

summary = summary.withColumn('percent_five_star', summary['five_star_ratings']/summary['all_ratings']*100)

summary = summary.select(['vine', 'all_ratings', 'five_star_ratings', 'percent_five_star'])

summary.show()

+----+-----------+-----------------+------------------+
|vine|all_ratings|five_star_ratings| percent_five_star|
+----+-----------+-----------------+------------------+
|   Y|         94|               48| 51.06382978723404|
|   N|      40471|            15663|38.701786464381904|
+----+-----------+-----------------+------------------+

